In [ ]:
%matplotlib inline
import time
import warnings

import corner
import emcee
import lmfit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import norm

warnings.filterwarnings("ignore")
plt.style.use("ggplot")

# JupyterHub benchmarking

## Test 6: Non-linear optimisation and curve fitting

**Based on one of the old DSToolkit tutorial notebooks**.

In this tutorial, we'll explore a synthetic example fitting the **double exponential** function:

$$y = a_1 exp \left(\frac{-x}{t_1}\right) + a_2 exp \left(-\frac{(x - 0.1)}{t_2}\right)$$

This is notoriously difficult to fit due to very sensitive parameter co-variances. It's an interesting test case because, even with just 4 parameters, it's surprisingly hard to optimise.

## 1. Generate test data

In [ ]:
def deterministic_model(params, x, obs=None):
    """Double exponential model. If no observations are passed, returns the model
    output. Otherwise, returns the residuals.
    """
    v = params.valuesdict()
    model = v["a1"] * np.exp(-x / v["t1"]) + v["a2"] * np.exp(-(x - 0.1) / v["t2"])

    if obs is None:
        return model

    return model - obs

In the code below, I've included an explicit **heteroscedastic** error term (i.e. $\sigma_{\epsilon} \propto y$), as this is relevant to the other tutorial on catchement modelling using [Mobius](https://github.com/NIVANorge/Mobius).

In [ ]:
# Number of data points
n_pts = 10000

# Fake 'true' params
true_params = lmfit.Parameters()
true_params.add("a1", value=3)
true_params.add("t1", value=2)
true_params.add("a2", value=-5)
true_params.add("t2", value=10)
true_params.add("m", value=0.05)

# Fake data
np.random.seed(0)
x = np.linspace(1, 10, n_pts)
y = deterministic_model(true_params, x)
m = true_params["m"].value
y += np.random.normal(scale=m * np.abs(y), size=n_pts)

# Plot
fig = plt.figure(figsize=(10, 6))
plt.plot(x, y)
plt.title("Synthetic raw data to fit")

## 2. Optimise

We start by creating a `Parameters` object and guessing some initial values for the **deterministic** model parameters.

In [ ]:
# Define initial params for optimisation
# Ignore error term at this stage
params = lmfit.Parameters()
params.add("a1", value=4.0)
params.add("t1", value=3.0)
params.add("a2", value=4.0)
params.add("t2", value=3.0)

The code below uses a two-step optimisation procedure for better results. Note that all these optimisers are performing least squares minimisation of the residuals, as returned by the `deterministic_model()` function (defined above). The great thing about this approch is that your deterministic model can be ***anything***. Here, we just have a "toy" Python function, but in the Mobius tutorial the deterministic Python function calls a catchment model (SimplyP), which is coded in C++. As long as your function returns the residuals (i.e. simulated - observed), the rest of the optimisation and MCMC code is basically unchanged, which saves a huge amount of time and effort.

In [ ]:
# Create minimisers
mi = lmfit.Minimizer(deterministic_model, params, fcn_args=(x, y), nan_policy="omit")

# First solve with Nelder-Mead (more robust than LM)
res = mi.minimize(method="nelder")

# Then solve with Levenberg-Marquardt using the Nelder-Mead solution
# as a starting point
res = mi.minimize(method="leastsq", params=res.params)

# Robustly estimate CI
ci = lmfit.conf_interval(
    mi,
    res,
    sigmas=[
        0.95,
    ],
)

# Calculate final result
final = res.residual + y  # Residual was (model - y), so this is an
# easy way of getting back to 'model'

# Report
lmfit.report_fit(res)
print("\n--------------------------\n")
lmfit.printfuncs.report_ci(ci)

# Plot
fig = plt.figure(figsize=(10, 6))
plt.plot(x, y)
plt.plot(x, final, "k")
plt.show()

Together, the optimisers do a reasonable job of recovering the "true" parameters, despite the very strong correlations.

## 3. MCMC

To get a more comprehensive understanding of the model parameters and their interactions, we can use Markov chain Monte Carlo (MCMC) sampling, optionally including "parallel tempering". If you're new to this topic, there's an introduction [here](http://jamessample.github.io/enviro_mod_notes/).

First, we define a **log-likelihood** function, which in this case is also our **log-posterior** (since we're using uniform priors). We also need to add the **stochastic** error parameter, `m`, to the deterministic parameter set we created above. Note that the error term gets added to the `params` of the *optimiser result* object (`res`), and not the original `params` object, because we want to start the MCMC from near the optimised location. 

In [ ]:
def lnprob(params, x, y):
    """Log-likelihood for simple Gaussian error structure. Because
    we're using uniform (improper) priors, this is also the
    log-posterior.
    """
    # Unpack sigma
    m = params["m"].value
    model = deterministic_model(params, x)
    sigma = m * np.abs(model)

    # Calculate log likelihood
    ll = np.sum(norm(model, sigma).logpdf(y))

    if np.isfinite(ll):
        return ll
    return -np.inf

In [ ]:
# We want to include sigma as an additional parameter in the MCMC
# Start from 'best fit' params found above
res.params.add("m", value=0.1, min=0.001, max=2)

In [ ]:
for i in range(10):
    start = time.time()

    # Run emcee sampler
    mi_mcmc = lmfit.Minimizer(lnprob, res.params, fcn_args=(x, y), nan_policy="omit")
    result = mi_mcmc.emcee(
        params=mi.params,
        burn=500,
        steps=1000,
        nwalkers=50,
        thin=20,
        ntemps=1,  # > 1 perform for Parallel Tempering
        float_behavior="posterior",  # lnprob returns a float representing log-posterior prob
    )

    end = time.time()
    elapsed = end - start
    print(f"{elapsed:.1f}")